In [1]:
%load_ext autoreload
%autoreload 2
from src import run_exp, statistics, get_graph, split_pos_neg, label_propagation
import networkx as nx
import pandas as pd
from pytorch_lightning import seed_everything
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
# baseline_methods = ['metis', 'louvain', 'eva', 'maxflow_cpp_unweighted', 'maxflow_cpp_weighted', 'gnn', 'pads_cpp']
baseline_methods = ['maxflow_cpp_weighted', 'neg_dsd_cpp', 'gnn', 'pads_cpp']#, 'pads_python_3k_2k', 'pads_python_4k_3k']
datasets = ['Abortion', 'Brexit', 'Election', 'Gun', 'Partisanship', 'Referendum_']
seed_everything(20010302)

Seed set to 20010302


20010302

In [9]:
theta = 0.3
baseline_methods = ['pads_cpp', 'high_degree']#, 'pads_python_3k_2k', 'pads_python_4k_3k']
for d in datasets:
    print(f"===Dataset {d}===")
    G = get_graph(d)
    # split_pos_neg(G, f'input/datasets/static/{d}', 500)
    timer = {}
    for method in baseline_methods:
        if method.startswith('pads_python_'):
            parts = method.split('_')
            if len(parts) == 4:
                size1 = int(parts[2].replace('k', '000'))
                size2 = int(parts[3].replace('k', '000'))
                timer[method], rst = run_exp(G, 'pads_python', dataset=d, theta=theta,
                                           size_ub=(size1, size2), attr_name=method)
        else:
            timer[method], rst = run_exp(G, method, dataset=d, theta=theta, binary=False)
        print(f"{method} Finished! Time taken: {timer[method]}")

    # G = label_propagation(G, attr_name='pads_cpp', new_attr='pads_cpp_lp', threshold=0.7)

    if not os.path.exists(f'output/results-theta={theta}/{d}/'):
        os.makedirs(f'output/results-theta={theta}/{d}/')
    nx.write_gml(G, f'output/results-theta={theta}/{d}/graph.gml')
    pd.DataFrame(timer.items(), columns=['method', 'time']).to_csv(f'output/results-theta={theta}/{d}/time.csv')

    df_pos, df_neg = statistics(G, f'output/results-theta={theta}/{d}')
    print(df_pos)
    print(df_neg)

===Dataset Abortion===
pads_cpp Finished! Time taken: 0.001716
G_pos: [1031, 1019, 3593, 11, 532, 3096, 32, 1062, 3112, 556, 3633, 3640, 2619, 3134, 4160, 580, 3654, 4168, 3663, 1615, 4182, 3163, 3681, 2151, 2664, 105, 3184, 2169, 639, 1664, 4224, 2691, 1161, 2187, 2189, 3727, 153, 1178, 2202, 2717, 1695, 3231, 2723, 168, 3755, 684, 3761, 698, 2235, 4284, 1725, 186, 3263, 2755, 4297, 1741, 206, 2259, 1751, 4311, 2778, 4319, 3296, 2272, 3811, 1763, 235, 3822, 2289, 2802, 3826, 1266, 3320, 1786, 255, 770, 2306, 3335, 3857, 3346, 2326, 3864, 1820, 797, 1309, 1834, 2348, 3886, 817, 1843, 1336, 3384, 826, 3898, 3391, 3394, 2886, 3402, 330, 2380, 3925, 353, 2927, 1394, 2420, 383, 2950, 3466, 3984, 912, 1428, 1946, 2459, 4513, 3491, 3496, 944, 1969, 1971, 1470, 3522, 451, 964, 3016, 2506, 3532, 3024, 4562, 2003, 979, 2519, 1496, 474, 4062, 3551, 3046, 2024, 489, 3574, 3065, 2555, 4093, 510], G_neg: [1281, 4098, 3715, 3076, 2050, 905, 3724, 2963, 4756, 277, 2835, 2712, 4248, 4122, 4255, 1190, 